<a href="https://colab.research.google.com/github/Krishnaugale353/SUmmaryOfMeeting/blob/main/Meeting_Summary_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import userdata
openai_key = userdata.get('OPENAI')
google_key = userdata.get('GOOGLE')

import os
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["GOOGLE_API_KEY"] = google_key

SecretNotFoundError: Secret OPENAI does not exist.

In [ ]:
##Business Need :
we need to automatically generate Minutes of Meeting (MoM) from meeting audio.
>> summary
>> discussed key-points
>> action items

In [ ]:
##Technical needs :
we'll get audio of meet :

create text from audio : speech to text  (Whisper)
generate (summary, key-points, action itmes) : LLm

##Creating Transcript :

In [ ]:
# Why Whisper Model :
>> SOTA in automatic speech recogniation (ASR) :
>> it support multiple languages
>> it is opensource (HuggigFace)
>> available through openai api

In [ ]:
## Why opensource (Huggingface) :
  -- since it is opensource, it is free to use
  -- almost as accurate as api
  -- no data sharing with third party (openai)


## Why openai API :
  -- less latency
  -- slightly better accuracy

In [ ]:
from IPython.display import Audio, display

display(Audio('/content/Council Meeting - July 19, 2022(1).mp3', autoplay=False))

###Transscript using HuggingFace :

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("automatic-speech-recognition",
                model="openai/whisper-large-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
display(Audio('/content/1013.mp3', autoplay=False))

In [ ]:
pipe("/content/sample-0.mp3")

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `

{'text': " He doesn't belong to you, and I don't see how you have anything to do with what is be his power yet. He's heaped us all in that from the stage to you."}

In [ ]:
pipe("/content/1013.mp3")

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': ' व्यक्ति को मातर इसलिए कि वह ज्यानी है सुईकार करने की अनुमती नहीं देते'}

###Transcript using opanai api

In [ ]:
!pip install -U openai

In [ ]:
# from openai import OpenAI

In [ ]:
from openai import OpenAI
client = OpenAI()


client.audio.transcriptions.create(
  model="whisper-1",
  file=open("/content/1013.mp3", "rb"),
  response_format="text"
)

'व्यक्ति को मात्र इसलिए कि वह घ्यानी है, सुईकार करने की अनुमती नहीं देते।\n'

In [ ]:
##Metric for ASR : Word error rate (WER)

##Minutes of Meeting :

In [ ]:
##Summary
##key points
##action items

In [ ]:
!pip install langchain langchain-community

In [ ]:
!pip install langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
from langchain.schema.messages import HumanMessage

In [ ]:
transcript = pipe("/content/Council Meeting - July 19, 2022(1).mp3")

In [ ]:
def extract_key_points(transcript):
  prompt = f"""You are highly skilled AI assistant in extracting key points
  from  meeting transcript. you will be be provided with
  meeting transcript and your job is to extract all the important key points discussed
  in meeting.
  transription : {transcript}"""

  message = HumanMessage(content= [
      {"type":"text", "text": prompt}
  ])

  response = model.invoke([message])
  return response.content

In [ ]:
with open("/content/transcription.txt", "r") as f:
  transcript = f.read()

In [ ]:
def create_summary(transcript):
  prompt = f"""You are highly skilled AI assistant in creating
  comprehensive summary from  meeting transcript. you will be be provided with
  meeting transcript and you job is to create concise summary in one paragraph.
  transription : {transcript}"""

  message = HumanMessage(content= [
      {"type":"text", "text": prompt}
  ])

  response = model.invoke([message])
  return response.content

In [ ]:
def extract_action_items(transcript):
  prompt = f"""You are highly skilled AI assistant in analuzing meetig conversations and
  extracting action items. analyze transcript carefully and indentify any task, assignments, deadlines
  or any actions agreed upon.
  you will be be provided with meeting transcript and you job is to
  extract action items
  transription : {transcript}"""

  message = HumanMessage(content= [
      {"type":"text", "text": prompt}
  ])

  response = model.invoke([message])
  return response.content

In [ ]:
def minutes_of_meeting(transcript):
  summary = create_summary(transcript)
  key_points = extract_key_points(transcript)
  action_items = extract_action_items(transcript)

  return {"summary":summary,
          "key points":key_points,
          "action items":action_items}

In [ ]:
minutes_of_meeting(transcript)

{'summary': 'The Rural Municipality of Springfield Council meeting began with the approval of the agenda and minutes from the previous meeting.  The council reviewed department reports, including the Animal Services report, which was deferred to the next meeting for further discussion. The consent agenda and list of accounts were approved, followed by the approval of the Springfield Centre security camera installation and the STARS Gala. The council authorized a one-year audit contract extension with Exchange Chartered Professionals Accountants, LLP and approved schedule changes for the upcoming meeting. The RCMP provided a quarterly report covering April to June, highlighting decreases in assaults and an increase in impaired driving charges. The council discussed staffing concerns within the RCMP, noting a shortage of officers due to transfers and health issues. The meeting concluded with a motion to adjourn. \n',
 'key points': "## Key Points from the Meeting Transcript:\n\n**1. Meet